<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
# Importing all the libraries

from datetime import datetime
import requests as r
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
from requests.exceptions import SSLError, Timeout, TooManyRedirects, RequestException
from contextlib import closing
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')
timeout_glob=2
verify_glob=True
allow_glob=False

# Defining the headers to access the webpage

headers = """accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
accept-encoding: gzip, deflate, br
accept-language: pt-PT,pt;q=0.9,en-US;q=0.8,en;q=0.7,fr;q=0.6,cs;q=0.5,es;q=0.4
cache-control: no-cache
cookie: __cfduid=d70a2e1775967b7e98c88e5e8e0e9a22b1603275849; _ga=GA1.2.306171135.1603275850; _gid=GA1.2.301121098.1603275850; adbrgn=FRIDF; mp_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjbGllbnROYW1lIjoid2liYml0eiIsImlucHV0TGFiZWwiOiJqc1RyYWNrZXIiLCJpbnB1dFR5cGUiOiJKU1NESyJ9.55Y_puhX1ed2hNYaK3G18PI5iuXXJJ_rEZSRVjfK7WE_alooma=%7B%22distinct_id%22%3A%20%221754ab0040e58d-05296d9c2063f9-c781f38-1fa400-1754ab0040f688%22%2C%22%24search_engine%22%3A%20%22google%22%2C%22%24initial_referrer%22%3A%20%22https%3A%2F%2Fwww.google.com%2F%22%2C%22%24initial_referring_domain%22%3A%20%22www.google.com%22%7D
pragma: no-cache
referer: https://www.politifact.com/
sec-fetch-dest: document
sec-fetch-mode: navigate
sec-fetch-site: same-origin
sec-fetch-user: ?1
upgrade-insecure-requests: 1
user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36"""
headers=dict([i.strip().split(': ') for i in headers.split('\n')])

# Defining the error handling functions

def printmd(string):
    display(Markdown('**'+string+'**'))
def is_good_response(x):
    """
    Returns True if the response seems to be an HTML, False otherwise
    x is response
    """
    content_type=x.headers['Content-Type'].lower()
    return ((x.status_code==200 and content_type is not None and content_type.find('html')>1), x.status_code)
def log_error(e):
    print(e)
def status_handler(status):
    if status<300:
        print('Alles gut!')
    elif status>=400 and status<500:
        print('request has failed due to you mistake bro. Check the link, headers, server and whether you have an access to this page.')
    else:
        print('Yoooo, I have no clue what has happened. But you are stuck, since you didnt receive any output from server')
    return None
def adaptive_scraping(url, timeout=timeout_glob, verify=verify_glob):
    global timeout_glob
    global verify_glob
    global allow_glob
    import requests as r
    try:
        with closing(r.get(url, stream=True, timeout=timeout_glob, verify=verify_glob, allow_redirects=allow_glob, headers=headers)) as resp:
            test1=is_good_response(resp)
            print(resp)
            if test1[0]:
                return resp.content
            else:
                return status_handler(test1[1])
    except Timeout as e:
        log_error(f'Error during request to {url}: {str(e)}')
        printmd('We can increase response waiting time. Do you want to continue?')
        ans=input('\n[Yes|No]\t')
        if ans.lower()=='yes':
            timeout_glob*=2
            return adaptive_scraping(url)
        else:
            return None
    except SSLError as e:
        log_error(f'Error during request to {url}: {str(e)}')
        printmd('We can skip verification test. Do you want to continue?')
        ans=input('\n[Yes|No]\t')
        if ans.lower()=='yes':
            verify_glob=False
            return adaptive_scraping(url)
        else:
            return None
    except TooManyRedirects as e:
        log_error(f'Error during request to {url}: {str(e)}')
        printmd('We can increase the number of allowed redirects. Do you want to continue?')
        ans=input('\n[Yes|No]\t')
        if ans.lower()=='yes':
            allow_glob=True
            return adaptive_scraping(url)
        else:
            return None
    except RequestException as e:
        log_error(f'Error during request to {url}: {str(e)}')
        printmd('Unfortunately, we have no clue what to do.  Please try again later. ')
        return None
    
# Defining the function to extract the information from the webpage

def extract():

    from datetime import datetime
    pages = [*range(1,1000)]
    ratings = []
    prices = []
    info = []
    name = []
    category = []
    discount = []
    stock = []

    # Creating the loop to web scrape each page

    for page in pages:
        url = 'https://www.darty.com/nav/achat/gros_electromenager/lave-linge/page'+str(page)+'.html'
        adaptive_scraping(url)
        response = r.get(url, headers=headers)
        soup = BeautifulSoup(response.content)
        container = soup.select('div.product_main_container')
        
        if container ==[]:
            break
        else:

            print('Retrieving data from page '+str(page))
            sleep(randint(2,6))

            # Append the information related to Ratings in the Ratings List

            for i in container:
                l=[i.text.strip() for i in i.select('span.rating_avis')]
                if len(l)==0:
                    ratings.append('0')
                else:
                    ratings.append(''.join(l))

             # Append the information related to Prices in the Prices List

            for i in container:
                l=[i.text.strip() for i in i.select('span.darty_prix')]
                if len(l)==0:
                    prices.append('0')
                else:
                    prices.append(''.join(l))

             # Append the information related to the features of the washer in the Info List

            for i in container:
                l=[i.text.strip() for i in i.select('ul.infos_strenghts')]
                if len(l)==0:
                    info.append('0')
                else:
                    info.append(''.join(l))

             # Append the information related to the category, brand and model in the Name List

            for i in container:
                l=[i.text.strip() for i in i.select('div.prd-family')]
                if len(l)==0:
                    name.append('0')
                else:
                    name.append(''.join(l))

             # Append the information related to Discounts in the Discounts List

            for i in container:
                l=[i.text.strip().replace('- ','') for i in i.select('span.striped_price')]
                if len(l)==0:
                    discount.append('0')
                else:
                    discount.append(''.join(l))

             # Append the information related to stock in the stock List

            for i in container:
                l=[i.text.strip() for i in i.select('div.status_delivery p.delivery_date')]
                if len(l)==0:
                    stock.append('0')
                else:
                    stock.append(''.join(l))
                
    # Merge every list into one dataframe

    df = pd.DataFrame(
        {'Name': name,
         'Info': info,
         'Price': prices,
         'Discount': discount,
         'Stock': stock,
         'Ratings': ratings
        })
    
    return df